In [2]:
import csv
import numpy as np
from sklearn import linear_model
import pandas as pd
import glob
import os

Stra="/home/ragu/Documents/"

df = pd.read_csv(Stra+"wok60.csv")
df = pd.DataFrame(df,columns=['v','i','p','s','q','pf','phi','Class'])
Str1="/home/ragu/Documents/voltage/1/"
Str2="/home/ragu/Documents/voltage/2/"
Str3="/home/ragu/Documents/voltage/3/"
Str4="/home/ragu/Documents/voltage/4/"
Str5="/home/ragu/Documents/voltage/5/"
Str6="/home/ragu/Documents/voltage/6/"

for (classin), group in df.groupby(['v','Class']):
    #print(group.head(10))
    if(classin[0]==190):
        group.drop(['v'], axis=1).to_csv(Str1+str(classin[1])+".csv",index=False,header=None)
    if(classin[0]==200):
        group.drop(['v'], axis=1).to_csv(Str2+str(classin[1])+".csv",index=False,header=None)
    if(classin[0]==210):
        group.drop(['v'], axis=1).to_csv(Str3+str(classin[1])+".csv",index=False,header=None)
    if(classin[0]==220):
        group.drop(['v'], axis=1).to_csv(Str4+str(classin[1])+".csv",index=False,header=None) 
    if(classin[0]==230):
        group.drop(['v'], axis=1).to_csv(Str5+str(classin[1])+".csv",index=False,header=None)
    if(classin[0]==240):
        group.drop(['v'], axis=1).to_csv(Str6+str(classin[1])+".csv",index=False,header=None)
        


In [3]:
from sklearn.neighbors import KNeighborsClassifier 
from sklearn import svm,cross_validation 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.naive_bayes import GaussianNB 
from sklearn import linear_model 
import pandas as pd 
import numpy as np 
import csv 
import os 
import time


dirname= "/home/ragu/Documents/"
num_of_folds=6
def traintestset(i,sdirname,trainData,trainTarget,testData,testTarget):
    for p in range(1,num_of_folds+1):
        dirnam = dirname+sdirname+str(p)+"/"
        if(p!=i):
            f2=open("/home/ragu/Documents/tr.csv",'a')
            for file1 in os.listdir(dirnam):
                f1=open(dirnam+file1,'r')
                temp=f1.read()
                f2.write(temp)
                f1.close()
            f2.close()
        else:
            f2=open("/home/ragu/Documents/te.csv",'a')
            for file1 in os.listdir(dirnam):
                f1=open(dirnam+file1,'r')
                temp=f1.read()
                f2.write(temp)
                f1.close()
            f2.close()

    df = pd.read_csv('/home/ragu/Documents/tr.csv',header=None)
    df=df.values
    trainData =df[:,:-1]
    trainData = trainData.astype(np.float)    
    trainTarget=df[:,-1:] 
    trainTarget = trainTarget.astype(np.int)
    trainTarget=trainTarget.T[0]
    df = pd.read_csv('/home/ragu/Documents/te.csv',header=None)
    df=df.values
    testData =df[:,:-1]
    testData = testData.astype(np.float)    
    testTarget=df[:,-1:]   
    testTarget = testTarget.astype(np.int)
    testTarget=testTarget.T[0]
    os.remove('/home/ragu/Documents/tr.csv')
    os.remove('/home/ragu/Documents/te.csv')
    return trainData,trainTarget,testData,testTarget

    
def fourfold(wcl):
    trainData = []
    trainTarget = []
    testData = []
    testTarget = []
    knna,svma,rfa,gnba,lra=0,0,0,0,0
    for i in range(1, num_of_folds+1):
        trainData,trainTarget,testData,testTarget=traintestset(i,wcl,trainData,trainTarget,testData,testTarget)
        #Apply standard learning algorithms on the data 
        
        start_time1 = time.time()
        knn = KNeighborsClassifier(n_neighbors=3) 
        knna=knna+knn.fit(trainData, trainTarget).score(testData, testTarget)
        #print("Duration1: %s secs" % (time.time() - start_time1))
        
        start_time1 = time.time()
        svc = svm.SVC()
        svma=svma+svc.fit(trainData, trainTarget).score(testData, testTarget)
        #print("Duration2: %s secs" % (time.time() - start_time1))
        
        start_time1 = time.time()
        clf = RandomForestClassifier(n_estimators=10)
        rfa=rfa+clf.fit(trainData, trainTarget).score(testData, testTarget) 
        #print("Duration3: %s secs" % (time.time() - start_time1))

        start_time1 = time.time()
        gnb = GaussianNB()
        gnba=gnba+gnb.fit(trainData, trainTarget).score(testData, testTarget)
        #print("Duration4: %s secs" % (time.time() - start_time1))

        start_time1 = time.time()
        logreg = linear_model.LogisticRegression(C=1e5)
        lra=lra+ logreg.fit(trainData, trainTarget).score(testData, testTarget) 
        #print("Duration5: %s secs" % (time.time() - start_time1))
        
    return knna,svma,rfa,gnba,lra


def printresults(knna,svma,rfa,gnba,lra,dd):
    print ("Six fold voltage wise cross validation accuracy of classifiers on full data "+dd )
    print ("KNN cv :",knna/num_of_folds )
    print ("SVM cv :",svma/num_of_folds )
    print ("RF cv :",rfa/num_of_folds )
    print ("GNB cv :",gnba/num_of_folds )
    print ("LR cv :",lra/num_of_folds)
    
    
knna,svma,rfa,gnba,lra=fourfold("voltage/")
printresults(knna,svma,rfa,gnba,lra,"voltage")



/home/ragu/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Six fold voltage wise cross validation accuracy of classifiers on full data voltage
KNN cv : 0.1652766568020805
SVM cv : 0.06972468836875617
RF cv : 0.44435476638866467
GNB cv : 0.46040713837324015
LR cv : 0.4577616357277374


In [4]:
from sklearn import linear_model
import pandas as pd
import glob
import os
import numpy as np
from math import sqrt
import csv 
import shutil
import time

class olsvimp():
    def __init__(self, strtyp,numofclass,dirname,num_of_folds):
        self.strtyp = strtyp
        self.numofclass = numofclass
        self.dirname = dirname
        self.num_of_folds = num_of_folds

    def traintestset(self,nthfold,sdirname,dirname):
        for p in range(1,num_of_folds+1):
            dirnam = dirname+sdirname+str(p)+"/"
            if(p!=nthfold):
                for file1 in os.listdir(dirnam):
                    path1=dirname+"olscv/train/"+str(nthfold)+"/"
                    if not os.path.exists(path1):
                        os.makedirs(path1)
                    with open(os.path.join(path1, file1),'a') as cf2:
                        writer = csv.writer(cf2)
                        cf1 = open(dirnam+file1)
                        csv_f = csv.reader(cf1)
                        for ro in csv_f:
                            writer.writerow(ro)                    
                        cf1.close()
            else:
                dirnam = dirname+sdirname+str(p)+"/"
                for file1 in os.listdir(dirnam):
                    path1=dirname+"olscv/test/"+str(nthfold)+"/"

                    if not os.path.exists(path1):
                        os.makedirs(path1)
                    with open(os.path.join(path1, file1),'a') as cf2:
                        writer = csv.writer(cf2)
                        cf1 = open(dirnam+file1)
                        csv_f = csv.reader(cf1)
                        for ro in csv_f:
                            writer.writerow(ro)                    
                        cf1.close()

    def fourfold(self):
        for nthfold in range(1, num_of_folds+1):
            self.traintestset(nthfold,strtyp,dirname)

    def euclidean_distance(self,row):
        """
        A simple manhattan distance function
        """
        self.inner_value = 0
        for k in self.distance_columns:
            self.inner_value += abs(row[k] - self.rownew[k]) #** 2
        return sqrt(self.inner_value)
    
    def mainalgo(self):
        self.fourfold()
        start_time1 = time.time()
        pd2=0                                             # counter to count the correctly identified classes in all folds
        for fold in range(num_of_folds):                             # 4 fold crossvalidation
            f1 = open(dirname+'myfile1.csv','w')  # create training file to store dimension pair regression summary
            for files in glob.glob(dirname+"olscv/train/"+str(fold+1)+"/*.csv"):  # from training files created from earlier program
                #print(files)
                df = pd.read_csv(files,sep=',')    
                x1= len(df.columns)
                df=df.values
                regr = linear_model.LinearRegression()                # apply regression model
                for i in range(1,x1):
                    for j in range(1,x1):
                        str1=""
                        if(i!=j and i<j):                             # on every dimension pair
                            #print(df[:,i-1:i], df[:,j-1:j])
                            regr.fit(df[:,i-1:i], df[:,j-1:j])
                            str1=str1+str(i)+" to "+str(j)+","+"%8.4f,%8.4f,%8.4f,"%(regr.score(df[:,i-1:i], df[:,j-1:j]),regr.coef_[0][0],regr.intercept_[0])
                            base=os.path.basename(files)
                            f1.write(str1+os.path.splitext(base)[0]+"\n")  # write regression result to file myfile1
            f1.close()      

            pd1=0                                               # counter to count the correctly identified classes for each fold
            print("fold"+str(fold+1))
            for fi2 in glob.glob(dirname+"olscv/test/"+str(fold+1)+"/*.csv"):
                base=os.path.basename(fi2)
                print(base)
                fi=os.path.splitext(base)[0]
            #for fi in range(1,numofclass):
                f2 = open(dirname+'myfile2.csv','w')   # create test file to store dimesion pair regression summary
                #df = pd.read_csv(dirname+"olscv/test/"+str(fold+1)+"/"+str(fi)+ ".csv",sep=',') # from test files created by earlier program
                df = pd.read_csv(fi2,sep=',') # from test files created by earlier program
                x1= len(df.columns)
                df=df.values
                regr = linear_model.LinearRegression()
                for i in range(1,x1):
                    for j in range(1,x1):
                        str1=""
                        if(i!=j and i<j):
                            regr.fit(df[:,i-1:i], df[:,j-1:j])
                            str1=str1+str(i)+" to "+str(j)+"," + "%8.4f,%8.4f,%8.4f,"%(regr.score(df[:,i-1:i], df[:,j-1:j]),regr.coef_[0][0],regr.intercept_[0])
                            f2.write(str1+fi+"\n")               # write test file dimesion pair regression summary
                f2.close()   

                # Match files to identify the class of test file
                myfile1_df=pd.read_csv(dirname+"/myfile1.csv",header=None) # match training file dimesion pair regression summary
                myfile1_df.columns=['ColA','ColB','ColC','ColD','ColE']
                self.predicted={}                                                         # dictionary to count votes of each class 
                
                f1a = open(dirname+"myfile2.csv")   

                for row in f1a:       # with test file dimesion pair regression summary
                    self.testrow = row.strip().split(",")
                    self.d={'ColB':float(self.testrow[1]),'ColC':float(self.testrow[2]),'ColD':float(self.testrow[3])}
                    self.rownew=pd.DataFrame(self.d,index=[1])
                    self.sel1= myfile1_df[myfile1_df['ColA'] ==  self.testrow[0]]               # retrieve same dimesion pair rows from myfile1
                    self.distance_columns=['ColB','ColC','ColD']
                    self.samedpairs_numeric = (self.sel1[self.distance_columns])
                    self.m_distance = self.samedpairs_numeric.apply(self.euclidean_distance, axis=1)      # apply manhattan distance
                    #self.m_distance = self.samedpairs_numeric.apply(lambda sqrt(self.inner_value): [self.inner_value += abs(row[k] - rownew[k]) for k in self.distance_columns], axis=1)
                    self.distance_frame = pd.DataFrame(data={"dist": self.m_distance, "idx": self.m_distance.index})
                    self.distance_frame.sort_values("dist", inplace=True)                               # strore and sort distances
                    self.most_similar = self.distance_frame.iloc[0]["idx"]
                    self.most_similar_class = self.sel1.loc[int(self.most_similar)]["ColE"]

                    self.secondmost_similar=self.distance_frame.iloc[1]["idx"]
                    self.secondmost_similar_class=self.sel1.loc[int(self.secondmost_similar)]["ColE"]

                    self.thirdmost_similar=self.distance_frame.iloc[2]["idx"]
                    self.thirdmost_similar_class=self.sel1.loc[int(self.thirdmost_similar)]["ColE"]

                    if(float(self.testrow[1])>0.1):                                               #fit value should be atleast >0.1 
                        self.predicted[self.most_similar_class]=self.predicted.get((self.most_similar_class), 0) + 1  # increase the vote using weigthed 3 neighbors
                        self.predicted[self.secondmost_similar_class]=self.predicted.get((self.secondmost_similar_class), 0) + 0.6  
                        self.predicted[self.thirdmost_similar_class]=self.predicted.get((self.thirdmost_similar_class), 0) + 0.4 

                print("predicted"+str(self.predicted))
                self.max_class=max(self.predicted, key=self.predicted.get)           # the class of test file is the class with maximum votes    
            
                if(int(fi)==self.max_class):
                    pd1=pd1+1                                          # Note the accuracy 
                    print("equal")
                    
                print(fi," ",self.max_class," ", pd1)
            pd2=pd1+pd2
            print("pd1"+str(pd1 ))
        print("pd2"+str(pd1 ))
        print ("Six fold voltage wise cross validation RBNN accuracy on full data for :",strtyp,(pd2*1.0/num_of_folds)/float(numofclass-1))                                   # crossvalidation accuray
        print("Duration6: %s secs" % (time.time() - start_time1))
        #shutil.rmtree(dirname+"olscv/train")
        #shutil.rmtree(dirname+"olscv/test")

dirname= "/home/ragu/Documents/"
num_of_folds=6
numofclass=64
strtyp="voltage/"
kk=olsvimp(strtyp,numofclass,dirname,num_of_folds)
kk.mainalgo()

dirname= "/home/ragu/Documents/"


fold1
112.csv
predicted{38: 2, 112: 4.2, 186: 2.0, 87: 1.4, 102: 1.6, 31: 4, 95: 2.4, 80: 3.0, 223: 5.199999999999999, 189: 1, 20: 1.2000000000000002, 23: 1, 193: 0.6, 83: 0.4}
112   223   0
20.csv
predicted{31: 2, 5: 2.2, 20: 3.4, 189: 0.4, 83: 1.6, 98: 3.4, 102: 5.200000000000001, 186: 2.2, 29: 1.7999999999999998, 3: 0.8, 293: 1.7999999999999998, 34: 1.2000000000000002, 80: 3, 193: 0.6, 23: 0.4}
20   102   0
31.csv
predicted{5: 1.6, 31: 7.0, 20: 1.7999999999999998, 26: 0.4, 98: 5.0, 34: 0.6, 95: 5.0, 83: 1.2, 189: 0.8, 384: 2.0, 273: 0.8, 19: 0.4, 293: 1, 25: 0.4, 131: 1, 16: 0.6, 256: 0.4}
equal
31   31   1
26.csv
predicted{20: 3, 80: 0.6, 26: 5.6, 31: 1.2, 223: 5, 8: 4.2, 112: 0.6, 186: 0.4, 38: 1.4, 143: 2.2, 263: 3, 19: 0.4, 4: 0.4, 75: 1, 29: 0.6, 78: 0.4}
equal
26   26   2
95.csv
predicted{80: 3, 26: 1.2, 95: 5.6, 75: 0.6, 98: 4.0, 31: 3.4, 83: 2, 112: 1.7999999999999998, 223: 1.2000000000000002, 34: 1, 384: 1.4, 273: 1.8, 19: 0.4, 293: 1.6, 25: 0.6, 62: 0.4}
equal
95   95   3


predicted{46: 3, 10: 2.2, 74: 0.4, 139: 3.4, 59: 4.0, 16: 1.0, 151: 1.7999999999999998, 154: 1.2000000000000002, 1: 1, 11: 0.6, 62: 0.6, 259: 0.4, 3: 0.4}
154   59   27
263.csv
predicted{131: 2.6, 263: 4.8, 256: 2.0, 384: 0.8, 143: 5.4, 158: 0.4, 98: 1.6, 34: 1.0, 31: 0.8, 4: 2, 19: 0.6, 8: 2.6, 26: 3, 112: 0.4, 48: 1, 57: 0.6, 151: 0.4}
263   143   27
131.csv
predicted{131: 6.6, 263: 4.4, 54: 0.8, 256: 1.2, 384: 0.4, 48: 4.800000000000001, 293: 0.6, 136: 0.8, 143: 4.0, 98: 0.4, 74: 1, 66: 1.2000000000000002, 158: 1.2000000000000002, 4: 1, 19: 0.6, 151: 0.6, 11: 0.4}
equal
131   131   28
53.csv
predicted{130: 5.400000000000001, 53: 7.6, 47: 5.4, 12: 6.199999999999999, 10: 1, 1: 2, 11: 0.8, 62: 1, 273: 0.6}
equal
53   53   29
74.csv
predicted{8: 1, 74: 11.4, 34: 0.4, 186: 0.6, 38: 0.4, 23: 1, 29: 0.4, 25: 3.8, 19: 1.2000000000000002, 13: 1.0, 11: 0.4, 263: 0.6, 4: 1.7999999999999998, 59: 1, 151: 0.6, 139: 0.4, 54: 0.4, 5: 0.6, 98: 0.6, 75: 0.4, 136: 1, 256: 0.6, 168: 0.4}
equal
74   74 

predicted{130: 9.0, 53: 5.6000000000000005, 47: 7.2, 57: 2, 134: 0.4, 12: 2.4, 4: 2, 1: 0.4, 259: 0.6, 139: 0.4}
equal
130   130   22
10.csv
predicted{46: 3.6, 47: 3.6, 12: 3.0, 10: 3.6, 11: 0.8, 1: 0.6, 139: 1.6, 259: 2.0, 62: 1.0, 154: 1.4, 130: 3.8, 53: 4.4, 3: 0.6}
10   53   22
384.csv
predicted{158: 1, 384: 7.200000000000001, 151: 1.0, 263: 1, 54: 0.4, 48: 0.4, 168: 4.0, 273: 9.0, 293: 0.8, 16: 1, 134: 1.4, 51: 0.6, 19: 0.6, 154: 1, 62: 0.6}
384   273   22
13.csv
predicted{143: 3.8000000000000003, 293: 0.6, 136: 1.4, 59: 1.4, 13: 8.4, 54: 5.8, 158: 1.7999999999999998, 48: 1.7999999999999998, 25: 0.6, 62: 1.6, 154: 0.8, 16: 0.6, 1: 1, 57: 0.4}
equal
13   13   23
18.csv
predicted{18: 8.2, 2: 4.8, 3: 1.2000000000000002, 134: 1.0, 26: 0.4, 11: 0.6, 5: 1.4, 19: 0.4}
equal
18   18   24
256.csv
predicted{256: 8.799999999999999, 54: 1.2, 263: 0.4, 131: 1.0, 151: 2.8, 59: 4.4, 136: 7.6, 168: 3.0, 273: 0.8}
equal
256   256   25
5.csv
predicted{5: 10, 31: 1.2, 20: 1.2000000000000002, 189: 0.

predicted{102: 3.0, 87: 4.6, 203: 0.4, 112: 0.6, 62: 1, 263: 4, 223: 2.6, 143: 2.0, 20: 2.1999999999999997, 78: 1.4, 23: 1.0, 75: 0.6, 193: 0.4, 158: 0.4, 186: 2.6, 80: 2.2, 18: 0.6, 8: 0.4}
equal
87   87   6
16.csv
predicted{189: 2, 78: 1.0, 98: 0.4, 31: 1, 26: 0.6, 20: 0.4, 5: 0.6, 62: 2.8000000000000003, 139: 0.6, 131: 0.8, 168: 2, 51: 1.0, 293: 2.2, 16: 6.4, 59: 1.4, 54: 1.7999999999999998, 13: 1.2000000000000002, 273: 1, 154: 0.8}
equal
16   16   7
4.csv
predicted{4: 13.2, 19: 3.8, 3: 0.8, 259: 1.0, 8: 0.4, 59: 0.4, 168: 0.6, 51: 1.4, 31: 0.4, 130: 0.6, 47: 0.4, 57: 0.4, 263: 0.6, 143: 0.4, 34: 1, 98: 0.4, 189: 1, 83: 0.4, 74: 1.2, 158: 1, 131: 0.6}
equal
4   4   8
59.csv
predicted{168: 5.4, 59: 12.2, 66: 2.0, 143: 0.4, 256: 1.4, 151: 4.2, 19: 0.6, 136: 1.0, 293: 1.2000000000000002, 74: 0.4, 154: 0.4, 139: 0.4, 131: 0.4}
equal
59   59   9
47.csv
predicted{53: 6.6000000000000005, 47: 8.999999999999998, 51: 0.4, 130: 8.799999999999999, 12: 2.4, 4: 2, 13: 0.4, 259: 0.4}
equal
47   47

predicted{223: 6.6, 203: 5.4, 102: 1.6, 139: 0.6, 193: 0.6, 26: 4.4, 263: 2.4, 38: 2.4, 112: 2.4, 80: 0.8, 189: 1, 143: 0.8, 2: 0.6, 18: 0.4}
equal
223   223   37
98.csv
predicted{98: 8.399999999999999, 29: 1.0, 83: 1.7999999999999998, 78: 0.6, 186: 2.8, 80: 0.6, 223: 1, 112: 0.4, 34: 1.7999999999999998, 31: 1.2000000000000002, 95: 1, 193: 4.8, 87: 1.6, 8: 2.6, 189: 0.4}
equal
98   98   38
189.csv
predicted{189: 7.2, 78: 1.2, 5: 1.2000000000000002, 20: 1.6, 8: 2.8000000000000003, 112: 1.7999999999999998, 29: 4.800000000000001, 102: 1, 31: 1.6, 34: 1.4, 223: 0.6, 80: 0.8, 83: 3, 87: 1}
equal
189   189   39
19.csv
predicted{19: 15, 4: 3.4000000000000004, 259: 1.0, 8: 0.6, 23: 1.0, 59: 0.8, 3: 0.4, 11: 0.6, 31: 1.0, 62: 1.2, 154: 0.4, 143: 0.6, 263: 0.4, 54: 0.6, 131: 0.4, 34: 0.6, 98: 0.4, 78: 0.4, 16: 0.4, 112: 0.4, 74: 0.4}
equal
19   19   40
11.csv
predicted{46: 4.0, 12: 3.6, 47: 0.4, 10: 4.2, 51: 0.8, 1: 3.0, 11: 3.6, 18: 0.4, 154: 2.0, 62: 1.6, 259: 0.4, 53: 3.4, 130: 0.6, 256: 1, 1

predicted{259: 3, 98: 0.6, 189: 1.0, 95: 1, 80: 0.6, 75: 0.4, 5: 0.4, 62: 6.4, 139: 2.2, 154: 4.8, 51: 3.4, 57: 1.6, 13: 0.4, 10: 0.4, 59: 0.4, 54: 0.4, 134: 1}
equal
62   62   26
3.csv
predicted{3: 12.0, 2: 3.6, 18: 0.8, 26: 0.6, 143: 0.4, 5: 0.6, 259: 0.4, 20: 1, 102: 1.0, 87: 1.4, 75: 0.6, 78: 2.0, 83: 0.4, 29: 0.6, 8: 0.4, 328: 0.8, 10: 1, 46: 0.4}
equal
3   3   27
193.csv
predicted{193: 10.799999999999997, 328: 0.6, 223: 1.4, 203: 1.0, 87: 8.8, 75: 2.9999999999999996, 78: 1.2, 112: 1.6, 80: 1, 20: 0.6}
equal
193   193   28
46.csv
predicted{46: 10.4, 12: 1.2, 47: 1.2000000000000002, 10: 7.399999999999999, 11: 2.4, 1: 1, 8: 0.6, 29: 0.4, 18: 1, 62: 0.4, 139: 1.6, 259: 1.6, 130: 0.4, 3: 0.4}
equal
46   46   29
8.csv
predicted{8: 10.799999999999999, 34: 1.6, 74: 0.4, 23: 1.0, 29: 2.4, 87: 0.4, 328: 4, 203: 2.2, 38: 3.8, 26: 1.2000000000000002, 112: 1.2000000000000002, 186: 0.6, 223: 0.4}
equal
8   8   30
186.csv
predicted{186: 7.799999999999999, 75: 1.0, 38: 1.6, 154: 0.4, 223: 5.8, 1

predicted{62: 1.6, 25: 0.6, 19: 1.4, 139: 1, 4: 1.0, 3: 0.4, 193: 3, 78: 1.2, 87: 1.4, 75: 0.4, 223: 1, 186: 0.6, 8: 0.4}
48   193   0
54.csv
predicted{62: 1.4, 19: 1.6, 25: 0.4, 139: 1, 4: 1.2, 3: 0.4, 193: 2, 78: 0.6, 87: 1.4, 75: 1.4, 23: 0.6, 223: 1, 8: 0.6, 203: 0.4}
54   193   0
151.csv
predicted{25: 1, 62: 1.2, 19: 1.4, 139: 1, 4: 1.0, 3: 0.4, 193: 3, 78: 1.2, 87: 1.4, 75: 0.4, 18: 2, 5: 1.2, 328: 0.8}
151   193   0
80.csv
predicted{75: 3.0, 186: 0.6, 95: 1.4, 80: 1.4, 154: 1, 16: 0.6, 259: 0.4, 193: 3, 78: 1.2, 87: 1.4, 18: 3, 5: 1.7999999999999998, 328: 1.2000000000000002}
80   75   0
29.csv
predicted{25: 1.4, 62: 1.6, 19: 1.4, 139: 1, 4: 0.6, 193: 2, 78: 0.6, 87: 1.0, 75: 0.4, 223: 1, 80: 0.6, 112: 0.4}
29   193   0
102.csv
predicted{193: 4.6, 328: 0.6, 223: 1.4, 203: 1.0, 112: 0.4, 78: 0.6, 87: 1.6, 75: 0.8, 23: 1}
102   193   0
83.csv
predicted{139: 2.6, 4: 2.2, 18: 1.0, 3: 0.8, 62: 1, 5: 1.6, 158: 0.4, 193: 1, 78: 0.6, 87: 0.4, 26: 0.4, 25: 1, 54: 0.6, 48: 0.4}
83   139   

predicted{5: 2, 31: 1.6, 189: 0.4, 20: 1.4, 26: 0.6, 193: 3, 78: 1.2, 87: 1.4, 3: 0.4}
95   193   1
136.csv
predicted{25: 1.6, 19: 1.6, 4: 1.4, 154: 1, 3: 0.8, 134: 1, 328: 2.6, 259: 0.4, 193: 3, 78: 1.2, 87: 1.4, 18: 1.2, 203: 0.8}
136   193   1
87.csv
predicted{87: 3.4, 102: 1.6, 203: 0.4, 112: 1.6, 193: 2, 78: 0.6, 3: 0.4, 5: 1, 18: 0.6, 26: 0.4}
equal
87   87   2
16.csv
predicted{19: 2, 25: 1.0, 4: 1.6, 154: 1, 3: 0.8, 134: 1, 259: 0.6, 51: 0.4, 193: 3, 78: 1.2, 87: 1.4, 328: 1, 18: 0.6, 203: 0.4}
16   193   2
4.csv
predicted{2: 5.0, 3: 2.4, 4: 1.2000000000000002, 154: 1, 1: 1, 46: 0.4, 11: 1, 328: 2.6, 18: 2.6, 203: 1.0, 186: 0.4, 19: 1, 57: 0.6, 26: 1, 13: 0.4, 8: 0.4}
4   2   2
59.csv
predicted{25: 1.4, 19: 1.6, 4: 1.6, 154: 1, 3: 0.8, 193: 3, 78: 1.2, 87: 1.4}
59   193   2
47.csv
predicted{25: 1.7999999999999998, 19: 1.6, 4: 1.4, 154: 1, 3: 3.6, 1: 1.4, 46: 1.2, 11: 1.4, 193: 4, 78: 1.2, 87: 1.4, 2: 6, 223: 1.0, 75: 0.4, 203: 1.0, 13: 0.4, 18: 0.6, 83: 0.6}
47   2   2
139.csv
p

predicted{19: 2, 25: 0.6, 4: 1.4, 154: 1, 3: 1.0, 2: 0.4, 134: 1.6, 259: 0.6, 51: 0.4, 193: 3, 78: 1.2, 87: 1.4, 328: 1, 18: 0.6, 203: 0.4, 95: 1, 29: 0.4}
11   193   7
203.csv
predicted{203: 2.6, 102: 1.2, 87: 1.7999999999999998, 193: 4, 112: 0.4, 223: 0.4, 78: 1.2, 3: 0.4}
203   193   7
158.csv
predicted{25: 1.4, 19: 1.6, 4: 2.0, 154: 0.6, 3: 0.8, 193: 3, 78: 1.2, 87: 1.4}
158   193   7
134.csv
predicted{25: 1.4, 19: 1.6, 4: 1.6, 154: 1, 3: 0.8, 134: 2, 51: 0.6, 259: 0.8, 328: 4.0, 193: 3, 78: 1.2, 87: 1.4, 143: 1, 54: 0.6, 18: 1.7999999999999998, 203: 1.2000000000000002, 223: 1, 186: 0.6, 8: 0.4}
134   328   7
12.csv
predicted{25: 1.7999999999999998, 19: 1.6, 4: 1.4, 154: 1, 3: 1.0, 1: 1.4, 26: 3.0, 46: 1.0, 11: 1, 193: 4, 78: 1.2, 87: 1.4, 2: 4.0, 13: 1.4, 328: 2.2, 18: 2.0, 83: 0.6}
12   193   7
328.csv
predicted{203: 2.6, 102: 1.2, 223: 1.2000000000000002, 193: 4, 78: 1.2, 87: 1.4, 3: 0.4}
328   193   7
38.csv
predicted{328: 2.6, 193: 3.6, 223: 1.7999999999999998, 203: 0.6, 78: 1